## Get bert-crf output

In [7]:
t_ner_path = '../bert-crf/bert-crf-model/test_predictions.pkl'

In [8]:
import pickle
from preproc import get_testing_data

from utils import timestamp

predictions = pickle.load(open(t_ner_path, 'rb'))
data = get_testing_data('../data/test_5.txt')
predictions = [p for prediction in predictions for p in prediction]

output = "article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
i = 0
for test_id in range(len(data)):
    pos = 0
    start_pos = None
    end_pos = None
    entity_text = None
    entity_type = None
    for pred_id in range(len(data[test_id])):
        if data[test_id][pred_id] == ' ':
            pos += 1
            continue

        if predictions[i][0] == 'B':
            start_pos = pos
            entity_type = predictions[i][2:]
        elif start_pos is not None and predictions[i][0] == 'I' and predictions[i+1][0] == 'O':
            end_pos = pos
            entity_text = ''.join([data[test_id][position][0]
                                    for position in range(start_pos, end_pos+1)])

            tokens = [str(test_id), str(start_pos), str(
                end_pos+1), entity_text, entity_type]
            line = '\t'.join(tokens)
            output += line + '\n'

            start_pos = None
        pos += 1
        i += 1
with open(f'outputs-tsv/output-crf.tsv', 'w', encoding='utf-8') as f:
    f.write(output)

## Merge bert and bert-crf outputs

In [9]:
import pandas as pd
bert_path = 'outputs-tsv/output-bert.tsv'
bert_crf_path = 'outputs-tsv/output-crf.tsv'

a_df = pd.read_csv(bert_path, sep='\t')
b_df = pd.read_csv(bert_crf_path, sep='\t')

a_df['src'] = 'bert'
b_df['src'] = 'bert-crf'
df = pd.concat([a_df, b_df.query("entity_type != 'ID'")])
df = df.drop_duplicates(subset=['article_id', 'start_position', 'end_position', 'entity_text', 'entity_type']).reset_index(drop=True)
df = df.sort_values(by=['article_id', 'start_position', 'end_position'])

In [10]:
start = df.groupby('article_id')['start_position'].apply(list)
end = df.groupby('article_id')['end_position'].apply(list)

for s, e in zip(start, end):
    for i, v in enumerate(e):
        s.insert(2*i+1, v) 

ret = pd.DataFrame() 
for idx, s in enumerate(start):
    for i in range(len(s)-1):
        if s[i] >= s[i+1]:
            start_pos = s[i+1]
            end_pos = s[i]
            temp = df.query("article_id == @idx and (start_position == @start_pos or end_position == @end_pos) and src == 'bert'")
            ret = ret.append(temp)

df.drop(ret.index, inplace=True)
df.drop(columns=['src']).to_csv('outputs-tsv/result.tsv', sep='\t', index=False)